In [ ]:
import shelve
import gdbm
mod = __import__('gdbm')
import dumbdbm
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import datasets, decomposition
import numpy as np
import gc
from datetime import datetime
import datetime as dt
#datetime.strptime('2016-03-29','%Y-%d-%m')
import dateparser
from ipykernel import kernelapp as app
import csv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from operator import itemgetter
import folium
from folium.plugins import HeatMap
import Instalib as IL
import math
import re
import string
import emoji
import pickle
import scipy.io
from sklearn.feature_extraction.text import CountVectorizer
import nltk.stem
import sktensor
import geopy.distance
from itertools import groupby
import ncp
from datetime import timedelta
from dateutil.relativedelta import *
import os

# Open the file and have a first look

Read the data and concatenate in a single DataFrame file, basic file format for pandas. It's a 2-D object, tabular file with labeled rows and columns, built from a dictionary or such. 

More info about it here: https://pandas.pydata.org/pandas-docs/stable/10min.html

Make our life easier by matching the Torino labels with the Marseille ones and convert strings to datetime

In [ ]:
df_new=pd.read_csv('/data/marcoq/instagram/torino/torino_posts.csv',lineterminator='\n')
df_new=df_new.rename(columns = {'caption':'text','taken_at_timestamp':'date'})
df_new.head()

In [ ]:
list_dates, Histo = IL.N_posts(df_new, debug=False)
Npostslog=[np.log10(H) for H in Histo]
Nposts=[H for H in Histo]

In [ ]:
if 'Figures' not in os.listdir('.') and False:
    os.mkdir('Figures')
    os.mkdir('Figures/Shape')
    os.mkdir('Figures/Shape/Spatial')
    os.mkdir('Figures/Shape/Temporal')
plt.figure()
plt.scatter(list_dates, Npostslog)
plt.xticks(list_dates[0::15])
plt.ylabel(r'$log_{10}$(#posts)')
plt.xlabel('Timeline')
plt.title(r'$log_{10}$ of the amount of insta posts per months')
#plt.savefig('Figures/Shape/Temporal/Torino_logN.pdf')
plt.show()
plt.figure()
plt.scatter(list_dates, Nposts)
plt.xticks(list_dates[0::15])
plt.ylabel('#posts)')
plt.xlabel('Timeline')
plt.title('Absolute amount of insta posts per months')
#plt.savefig('Figures/Shape/Temporal/Torino_N.pdf')
plt.show()

In [ ]:
df_location_all=pd.read_csv('/data/marcoq/instagram/torino/torino_locations.csv', lineterminator='\n')

In [ ]:
df_location_all=df_location_all.rename(columns = {'latitude':'lat','longitude':'lon', 'location_id':'id'})
df_location_all.head()

In [ ]:
#remember zip:
zip([1,2,3],['a','b','c'])
#it's a function.
#make a new column 'coord' indexing (lat,lon) for each post
df_location_all['coord'] = list(zip(df_location_all.lat,df_location_all.lon))
df_location_all.head()

In [ ]:
#Heatmap per year
TbyCoord=IL.time_coord(df_location_all, df_new)
for year in range(2010,2019,1):
    Coordinates=[]
    for t in TbyCoord:
        if t[0][0:4]==str(year):
            Coordinates.append(t[1])
    IL.geo_heatmap(Coordinates, save=False,filename='hm'+str(year))

# Let's refine the DataFrame

In [ ]:
#link postID with its location
dict_id_coord= dict(zip(df_location_all.id,df_location_all.coord))
dict_id_name= dict(zip(df_location_all.id,df_location_all.name))
dict_coord_name= dict(zip(df_location_all.coord,df_location_all.name))

series.map() matches corresponding entries from two different series files. This is what x.map(y) does:

In [ ]:
x = pd.Series([1,2,3], index=['one', 'two', 'three'])
x

In [ ]:
y = pd.Series(['foo', 'bar', 'baz'], index=[1,2,3])
y

In [ ]:
x.map(y)

In [ ]:
#Adding a column 'coords' by mapping the coordinates to their location_id:
df_new['coords'] = df_new['location_id'].map(dict_id_coord)
df_new['name'] = df_new['location_id'].map(dict_id_name)

#Create lat and lon columns by reading coords
# Look here: https://pythonhow.com/accessing-dataframe-columns-rows-and-cells/
df_new['lat']=[e[0] for e in df_new.loc[:,'coords']]
df_new['lon']=[e[1] for e in df_new.loc[:,'coords']]

# Drop duplicates
df_new.drop_duplicates(keep='first')
df_new.drop('location_id',axis=1,inplace=True)
df_new.head()

In [ ]:
#needs to be done at the end for compatibility reasons
# learn how to parse time here: https://www.tutorialspoint.com/python/time_strptime.htm

df_new['date']=[datetime.strptime(t,'%Y-%m-%d %H:%M:%S') for t in list(df_new['date'])]

In [ ]:
df_new.dtypes

### Checkpoint 1

In [ ]:
#pickle.dump(df_new, open('df_new_chkpt1.pkl', 'wb'))

Build the frame and the grid. $$x_{min/max}=longitude_{min/max}$$ and conversely for y and latitude.

In [ ]:
imp.reload(IL)

In [ ]:

# Attention problème d'arrondi lors du passage de Coord à (lat,lon)
step_m=0.5
lat1,lat2,step_lat,lon1,lon2,step_lon,to_bin_lon,to_bin_lat = IL.build_grid(df_new,step_m)
number_lats,number_lon,df_new=IL.add_grid_todf(df_new,to_bin_lat,to_bin_lon,step_lat,step_lon)
print (number_lats, number_lon)


### Checkpoint 2

In [ ]:
#pickle.dump(df_new, open('df_new_chkpt2.pkl', 'wb'))
#df_new=pd.read_pickle('df_new_chkpt2.pkl')

# Time to clean the text

But first make a copy before we do anything stupid

In [ ]:
df_new_save=df_new.copy()
#df_new=df_new_save.copy()

In [ ]:
#Purge all the newline characters
df_new['text']=df_new.text.replace('\n',' ',regex=True)
print df_new['text'][22].decode('utf-8')

In [ ]:
len(df_new.text.tolist())

This methods works only on unicode text. So we need to use the function x.decode('utf-8') to convert all strings in the 'text' field to unicode objects.

In [ ]:
text_0=df_new['text'][22][0:11].decode('utf-8')
print text_0

In [ ]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
#df_new['text_new2'] = df_new['text_new2'].map(lambda x: emoji_pattern.sub(r'', x))
       
#It works on unicode but not on a string:
    
text = u'This dog \U0001f602'
print(text) # with emoji
print(emoji_pattern.sub(r'', text)) # no emoji

text = 'This dog \U0001f602'
print(text) # with emoji
print(emoji_pattern.sub(r'', text)) # no emoji

print '\n but some emojis still pass through... \n'

text=text_0
print(text) # with emoji
print(emoji_pattern.sub(r'', text)) # no emoji

print(text) # with emoji
print(emoji_pattern.sub(r'', text)) # no emoji


Another way to deal with emoji would be to de-emojize them. They would then count as words, that can either be removed
since they have the very peculiar following form :grinning_face: 

The difficulty here is identifying emojis in a text. Their classification is hard to decypher

In [ ]:
print(text) # with emoji
print emoji.demojize(text) # no emoji

In [ ]:
#Remove all blank fields and encode everything in utf-8 if it is not already the case
df_new=df_new[df_new['text'].notnull()]
if not type(df_new['text'][0])==unicode:
    df_new['text']=df_new['text'].map(lambda x: x.decode('utf-8'))

In [ ]:
#remove emojis? some of them still pass through
#re.sub(a,b,string) = remplace le premier a par b dans string et renvoie string. 
#Si a n'est pas dans string, renvoie string.
keys=emoji.unicode_codes.UNICODE_EMOJI.keys()
df_new['text_new'] = df_new['text'].map(lambda x: emoji_pattern.sub(r'', x))
#df_new['text_new'] = df_new['text'].map(lambda x: rem_emj(x,keys)) 

In [ ]:
df_new=df_new[df_new['text'].notnull()]
if not type(df_new['text'][0])==unicode:
    df_new['text']=df_new['text'].map(lambda x: x.decode('utf-8'))

In [ ]:
#remove hyperlinks
df_new['text_new'] = df_new['text_new'].map(lambda x: re.sub(r'http\S+', '', x))

In [ ]:
#remove quotes
df_new['text_new'] = df_new['text_new'].map(lambda x: re.sub(r'&amp;quot;|&amp;amp', ' ', x))

In [ ]:
#Remove tags
df_new['text_new'] = df_new['text_new'].map(lambda x: re.sub(r'@[a-zA-Z0-9]*', '', x))

In [ ]:
#Remove tickers
df_new['text_new']= df_new['text_new'].map(lambda x: re.sub(r'\$[a-zA-Z0-9]*', '', x))

In [ ]:
#Remove numbers
df_new['text_new']=df_new['text_new'].map(lambda x:  re.sub(r'[0-9]*','',x))

In [ ]:
#Remove #
df_new['text_new']=df_new['text_new'].map(lambda x:  re.sub(r'#*','',x))

In [ ]:
#Remove punctuation
df_new.text_new = [filter(lambda x: x not in string.punctuation,tweet) for tweet in df_new.text_new]

In [ ]:
#print len(df_new.text_new)
df_new.head(10)

In [ ]:
#encoding back in strings
df_new['text_new'] = df_new['text_new'].map(lambda x: x.encode('utf-8'))

In [ ]:
df_new=df_new[df_new['text_new'].notnull()]
df_new=df_new[df_new['text_new'] != '']
df_new=df_new[['date','lat','lon','latbin','lonbin','coords','text_new','name']]
df_new=df_new.rename(columns = {'text_new':'text'})
df_new['coordsbin'] = list(zip(df_new.latbin,df_new.lonbin))
df_new.head()

In [ ]:
df_new=IL.text_processing(df_new)
df_new=df_new[['date','lat','lon','latbin','lonbin','coords','text_new','name']]
df_new=df_new.rename(columns = {'text_new':'text'})
df_new['coordsbin'] = list(zip(df_new.latbin,df_new.lonbin))

### Checkpoint 3

In [ ]:
#And pickle it
#df_new.to_pickle('df_new_chkpt3.pkl')
df_new=pd.read_pickle('df_new_chkpt3.pkl')
df_new=df_new[df_new['text'] != '']

In [ ]:
import imp

In [ ]:
if True:
    #In case I'm not loading this since the beginning
    step_m=0.5
    lat1,lat2,step_lat,lon1,lon2,step_lon,to_bin_lon,to_bin_lat = IL.build_grid(df_new,step_m)
    number_lats,number_lon,df_new=IL.add_grid_todf(df_new,to_bin_lat,to_bin_lon,step_lat,step_lon)

In [ ]:
df_new.head()

In [ ]:
dict_coord_name= dict(zip(df_new.coords,df_new.name))
dict_name_coord= dict(zip(df_new.name,df_new.coords))

Learn about how groupby does not returns a dataframe but only groups the data IN ORDER TO APPLY SOME FUNCTION to it here: https://pandas.pydata.org/pandas-docs/stable/groupby.html

In [ ]:
imp.reload(IL)
Degenerate_loc, Degenerate_names, Degenerate_locbin, Degenerate_namesbin,Degenerate_freq = IL.name_id_degeneracy(df_new)
Deg_names_dist = IL.pairwise_distances(Degenerate_loc,Degenerate_names)

In [ ]:
# How many places have associated coordinates further apart than an arbitrary threshold t?
# What are those places and what are their locations?

t=.7 #threshold in km
ct=0
for d in Deg_names_dist:
    if any( e>t for e in d[1]):
        ct+=1
        print d[0], '&', Degenerate_loc[Deg_names_dist.index(d)],'&', np.mean(d[1]),'\\\\'
print ct,'places above this distance threshold in a list of ', len(Deg_names_dist)

In [ ]:
# All locations associated with a degenerated place
Degloc_all=[]
for p in Degloc_copy:
    for d in p:
        Degloc_all.append(d)

In [ ]:
# Link the locations associated to a place in particular to visualise what it means.
m = folium.Map(location=[np.mean([c[0] for c in Degloc_all]),np.mean([c[1] for c in Degloc_all])], zoom_start=12)
#for p in Degloc_all:
 #   folium.Marker(p).add_to(m)
for p in Degloc_copy:
    p.append(p[0])
    folium.PolyLine(locations=p).add_to(m)
m
#m.save('places_hiN.html')

In [ ]:
imp.reload(IL)

In [ ]:
N_fdate,dict_places_n=IL.first_apparition(df_new)
    
plt.figure()
plt.scatter([n[1] for n in N_fdate],[n[0] for n in N_fdate])
plt.xlabel('date of first apparition')
plt.ylabel(r'$N_{occurences}$')
#plt.savefig('N_data_first_apparition.pdf')
plt.show()

In [ ]:
Outliers=[]
# Who are the outliers of the two last years (The places with a number of posts too high)
# But not extremely high.
for n in N_fdate:
    if n[1].year>=2016:
        if n[0]>=2000 and n [0]<10000:
            Outliers.append((n[2],n[0]))
    if n[1].year>=2017:
        if n[0]>=1500 and n [0]<2000:
            Outliers.append((n[2],n[0]))
Outliers=[(e[0], e[1], dict_name_coord[e[0]]) for e in Outliers]
Outliers

In [ ]:
Places_to_get=[]
# How to access the points which are out of the general tendency (monuments, etc. ?)
for k in dict_places_n.keys():
    if dict_places_n[k]>10000:
        Places_to_get.append([dict_name_coord[k],dict_places_n[k]])

#Places_to_get is a list of [(lat,lon), N_post] for the places with a unusually high N_post.

In [ ]:
IL.plot_time_series(Places_to_get,dict_coord_name,df_new,'filename',False)

In [ ]:
# Where are the places that gather so much posts, on a map?
m = folium.Map(location=[np.mean([c[0][0] for c in Places_to_get]),np.mean([c[0][1] for c in Places_to_get])], zoom_start=12)
for p in Places_to_get:
    folium.Marker(p[0]).add_to(m)
m
#m.save('places_hiN.html')

In [ ]:
print len(set(df_new_nodup_cells.coords.tolist())), 'different coordinates in the dataset'

In [ ]:
df_new_index,df_new_nodup2=IL.indexed_dataframe(df_new,step_lon,step_lat,number_lats)

In [ ]:
#Make an index of only the years + months and text of posts
df_new_index=df_new[['date','text','coordsbin']]
df_new_index['date'] = pd.to_datetime(df_new_index['date'], errors='coerce')
df_new_index['year_month'] = list(zip(df_new_index['date'].dt.year, df_new_index['date'].dt.month)) #add a month column
yearmin=min([y[0] for y in df_new_index.year_month.tolist()])
df_new_index['month_abs']=[(e[0]-yearmin)*12+e[1] for e in df_new_index.year_month]

In [ ]:
print df_new_index.shape
print len(set(df_new_index.text.tolist()))
print len(df_new_index.text.tolist())
print len(set(df_new_index.coordsbin.tolist()))
print len(df_new_index.coordsbin.tolist())

### Checkpoint 4

In [ ]:
#And pickle it
#df_new_index.to_pickle('df_new_index_chkpt4.pkl')
df_new_index=pd.read_pickle('df_new_index_chkpt4.pkl')
df_new_index=df_new_index[df_new_index['text'] != '']

In [ ]:
list_years=list(set([e[0] for e in df_new_index.year_month.tolist()]))
list_months=list(set([e[1] for e in df_new_index.year_month.tolist()]))
print sorted(list_years)
print sorted(list_months)

In [ ]:
start=datetime(2010,10,1)
stop=datetime(2018,12,12)
flags=IL.TimeFlags(start,stop,'months',1)
slices=[(f,f+relativedelta(months=1)) for f in flags]

In [ ]:
# My version NEW
df_classified=IL.dataframe_classified(slices,df_new_index)
df_classified.head()

### Checkpoint 5

In [ ]:
#And pickle it
#df_new.to_pickle('df_new_chkpt5.pkl')
#df_new_index.to_pickle('df_new_index_chkpt5.pkl')
#df_classified.to_pickle('df_classified_chkpt5.pkl')

#df_new=pd.read_pickle('df_new_chkpt5.pkl')
#df_new_index=pd.read_pickle('df_new_index_chkpt5.pkl')
#df_classified=pd.read_pickle('df_classified_chkpt5.pkl')


In [ ]:
if 1:
    lat1,lat2,step_lat,lon1,lon2,step_lon,to_bin_lon,to_bin_lat = IL.build_grid(df_new,step_m)
    number_lats,number_lon,df_new=IL.add_grid_todf(df_new,to_bin_lat,to_bin_lon,step_lat,step_lon)
    print (number_lats, number_lon)
    #Make an index of only the years + months and text of posts
    gc.collect()
    list_date=list(set([e.date() for e in df_new.date.tolist()]))
    list_years=list(set([e.year for e in list_date]))
    list_months=list(set([e.month for e in list_date]))
    df_new_index=df_new[['date','text','coordsbin']]
    df_new_index['date'] = pd.to_datetime(df_new_index['date'], errors='coerce')
    df_new_index['year_month'] = list(zip(df_new_index['date'].dt.year, df_new_index['date'].dt.month)) #add a month column
    yearmin=min([y[0] for y in df_new_index.year_month.tolist()])
    df_new_index['month_abs']=[(e[0]-yearmin)*12+e[1] for e in df_new_index.year_month]

In [ ]:
# Let's group the data by month and count how many coordsbin we have. 

# Group by month:
count_Ncells_time=df_new_index[['month_abs','coordsbin']].set_index('coordsbin').groupby('month_abs')
# Access the dictionary of groups (groups are the keys, coords are the values):
dict_month_Ncells=count_Ncells_time.groups
# For each month, let's count how many cells are filled.
list_coords_Nposts=[]
for month in dict_month_Ncells.keys():
    coords=dict_month_Ncells[month]
    # We want to keep only one occurence for each cell grid.
    list_coords_Nposts.append((month, len(set(coords))))


In [ ]:
dates_Ncells=[IL.abs_to_yearmonth(m[0]) for m in list_coords_Nposts]

In [ ]:
reload(IL)
for gridstep in [.5,1,3,5]:
    plt.figure()
    dates_Ncells_1, list_coords_Nposts_1 = IL.N_cells_info(df_new,gridstep)
    plt.plot(dates_Ncells_1,[e[1] for e in list_coords_Nposts_1])
    plt.xlabel('Month')
    plt.ylabel(r'$N_{cells}$ filled')
    plt.savefig('N_gridcells_step='+str(gridstep)+'.pdf')
    plt.show()
    
    steplon=gridstep/(40000*math.cos((lat1+lat2)*math.pi/360)/360)
    steplat=gridstep/(40000./360.)

for gridstep in [.5,1,3,5]:
    lng=lon1
    lonbins=[lng]
    while lng<lon2+steplon:
        lng+=steplon
        lonbins.append(lng)

    lat=lat1
    latbins=[lat]
    while lat<lat2+steplat:
        lat+=steplat
        latbins.append(lat)
    continue
        # Link the locations associated to a place in particular to visualise what it means.
    m = folium.Map(location=[np.mean(latbins),np.mean(lonbins)], zoom_start=12)
    for lng in lonbins:
        folium.PolyLine(locations=((min(latbins),lng),(max(latbins),lng)),weight=1).add_to(m)
    for lat in latbins:
        folium.PolyLine(locations=((lat,min(lonbins)),(lat,max(lonbins))),weight=1).add_to(m)
    #m.save('grid_'+str(gridstep)+'.html') 

# Term vector building by NMF

Stemming = removing all suffixes in the word to keep only its root.

This part executes NMF on time slices of the data, to have an idea of what the output might be. It generates .mat files that can be used by matlab.

More about NMF here: https://scikit-learn.org/stable/modules/decomposition.html#nmf

In [ ]:
from nltk.corpus import stopwords
french_stopwords = set(stopwords.words('french')) #le la les ou å pour etc.
english_stopwords = set(stopwords.words('english'))
italian_stopwords = set(stopwords.words('italian'))
from nltk.stem.snowball import FrenchStemmer
import nltk.data #
nltk.download('punkt')
from nltk.stem.snowball import FrenchStemmer
#Filter out the monuments?
monuments=False
all_stopwords=french_stopwords.union(english_stopwords)
all_stopwords=all_stopwords.union(italian_stopwords)
all_stopwords=all_stopwords.union(IL.words_added)
all_stopwords=all_stopwords.union(IL.words_filtered)
if monuments:
    all_stopzords=all_stopwords.union(IL.words_monuments)
    
italian_stemmer = nltk.stem.SnowballStemmer('italian')
list_reviews_all=df_classified.result.tolist()
n_features=1000
#Build the vectorizer and the list of all words in the dataset
#df_new=pd.read_pickle('Tor_instagram_2.pkl')
vectorizer_s = IL.StemmedCountVectorizer(max_df=0.95, max_features=n_features, analyzer="word", stop_words=all_stopwords)
while '' in list_reviews_all:
    list_reviews_all.remove('')
while "" in list_reviews_all:
    list_reviews_all.remove("")

More info about countvectorizer here: 
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

And about TfIdf: 
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html

And about NMF:
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html

In [ ]:
#pickle.dump(df_classified, open('df_granularity.pkl','wb'))

In [ ]:
#counts = vectorizer_s.fit_transform(list_reviews_all)
#pickle.dump(counts, open('counts_master.pkl','wb'))
#pickle.dump(vectorizer_s, open('vec_s_master.pkl','wb'))
counts = pickle.load( open( 'counts_master.pkl', "rb" ) )
vectorizer_s = pickle.load( open( 'vec_s_master.pkl', "rb" ) )

In [ ]:
# Feed the vectorizer with all the words in the dataset. Counts is the tweet/term matrix.
# fit_transform: fit first (build the features list with the relevant words)
# then transform: build the tweet/term matrix with the relevant tokens.

vectorizer_new = StemmedCountVectorizer(max_df=0.95, max_features=n_features, analyzer="word", stop_words=all_stopwords,vocabulary=vectorizer_s.vocabulary_) 
voc_vector={k:v for v,k in vectorizer_s.vocabulary_.iteritems()}
voc_serie=pd.Series(voc_vector)

In [ ]:
vectorizer_new = StemmedCountVectorizer(max_df=0.95, max_features=n_features, analyzer="word", stop_words=all_stopwords,vocabulary=vectorizer_s.vocabulary_) 
voc_vector={k:v for v,k in vectorizer_s.vocabulary_.iteritems()}
voc_serie=pd.Series(voc_vector)
#F = open('termvectors_sampling_5topics.txt','a') 
start=20
n_topics=15
for sample_run in xrange(1):
    A,B,C,TermVectorsIndex,TermVectors = IL.NTF_sampling(False,df_classified,flags,0,0,vectorizer_new=vectorizer_new,n_topics=n_topics)
    for tvi in TermVectorsIndex:
        print TermVectorsIndex.index(tvi), [t[0] for t in tvi[0:8]]
#        F.write(str([t[0] for t in tvi[0:10]]))
#        F.write('\n')
#F.close()

In [ ]:
#Topics definition
plt.figure(figsize=(10,4))
plt.pcolormesh(B.T)#, cmap=plt.cm.Greys)
plt.xlim(0,B.shape[0])
plt.yticks(np.arange(B.shape[1])+0.5, np.arange(1,B.shape[1]+1))
#plt.savefig('Topic_definition_normalized.pdf')
plt.show()

#for tvi in TermVectorsIndex:
    #print TermVectorsIndex.index(tvi), [t for t in tvi[0:3]]
#    print TermVectorsIndex.index(tvi), [t[0] for t in tvi[0:5]]

In [ ]:
CommonWords=set([])
for t in TermVectors:
    for t_other in TermVectors:
        if t_other!=t:
            for term in t.union(t_other):
                CommonWords.add(term)

In [ ]:
for t in TermVectors:
    T=list(t)
    print len(t)
    for word in t:
        if word in CommonWords:
            T.remove(word)
    t=set(T)
    print len(T)

In [ ]:
[]
for tvi in TermVectorsIndex:
    #print TermVectorsIndex.index(tvi), [t for t in tvi[0:3]]
    print TermVectorsIndex.index(tvi), [t[0].encode('utf-8') for t in tvi[0:10]]

In [ ]:
#Topics by bin
plt.figure(figsize=(10,4))
plt.pcolormesh(A.T)#, cmap=plt.cm.Greys)
plt.xlim(0,A.shape[0])
plt.yticks(np.arange(B.shape[1])+0.5, np.arange(0,B.shape[1]+1))
#plt.savefig('Topic_bybin_normalized.pdf')
plt.show()

for tvi in TermVectorsIndex[::-1]:
    #print TermVectorsIndex.index(tvi), [t for t in tvi[0:3]]
    print TermVectorsIndex.index(tvi), [t[0] for t in tvi[0:5]]

In [ ]:
for index in range(0,len(A.T)):
    row=A.T[index]
    print 'topic',index, '[',min(row),';', max(row)*100,']'

In [ ]:
#Geographical repartition of topics
for index in range(0,len(A.T)):
    row=A.T[index]
    Coordinates=list(df_classified.index)
    x=sorted(set([Cd[0] for Cd in Coordinates]))
    y=sorted(set([Cd[1] for Cd in Coordinates]))
    z=[(Coordinates[i], row[i]) for i in range(0,len(row))]
    hem=np.zeros((len(x),len(y)))
    for e in z:
        hem[x.index(e[0][0])][y.index(e[0][1])]=e[1]
    hmap = folium.Map(location=[np.mean(x), np.mean(y)], zoom_start=10, )

    hm_wide = HeatMap( list(zip([C[0] for C in Coordinates],[C[1] for C in Coordinates],row)),
                       min_opacity=0,
                           max_val=max(row),
                       radius=15, blur=0, 
                       max_zoom=1, 
                     )
    hmap.add_child(hm_wide)

    hmap.save('Heatmaps/heatmap_topic'+str(index)+'_normalized.html')

In [ ]:
#Topics Dynamics
plt.figure(figsize=(10,4))
plt.pcolormesh([c[start:] for c in C.T])#, cmap=plt.cm.Greys)
plt.xlim(0,len(C.T[0])-start)
#plt.savefig('Topic_dynamic.pdf')
plt.yticks(np.arange(n_topics)+0.5, np.arange(0,n_topics));
#plt.xticks()
plt.show()
for tvi in TermVectorsIndex[::-1]:
    #print TermVectorsIndex.index(tvi), [t for t in tvi[0:3]]
    print TermVectorsIndex.index(tvi), [t[0] for t in tvi[0:5]]

In [ ]:
os.mkdir('sample_timelines')
for index in range(0, len(C.T)):
    row=C.T[index]
    plt.figure()
    plt.plot([abs_to_yearmonth(i) for i in range(0,len(row))], row)
    plt.title('relative intensity of topic '+str(index)+' in a global monthly timeline')
    plt.savefig('sample_timelines/plot_topic'+str(index)+'.pdf')
    plt.show()

In [ ]:
dfa = pd.DataFrame(A).groupby(voc_serie).sum()
#dfa = dfa[dfa.index!='teachers']

In [ ]:
dfa

In [ ]:
plt.figure(figsize=(10,4))
plt.pcolormesh(C.T)#, cmap=plt.cm.Greys)
plt.xlim(0,C.shape[0])
plt.yticks(np.arange(14)+0.5, np.arange(1,15));

In [ ]:
import scipy.io
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.io import savemat
n_features=1000
n_topics=10

list_reviews_all=df_new['text'].tolist()
counts = vectorizer_s.fit_transform(list_reviews_all)
vectorizer_new = CountVectorizer(max_df=0.95, max_features=n_features,stop_words=all_stopwords_add,vocabulary=vectorizer_s.vocabulary_)
ct=0

# For each month in the dataset, get a snapshot view of the NMF tensor:
for month in range(1,(len(list_years)-1)*12+2,3):
    
    year_start=list_years[0]+ int(month/12)
    month_start=month%12  
    year_end=list_years[0]+ int(month+11-0.5)/12
    month_end=1+(month+11-1)%12
    
    try:
        list_reviews_rest=dataframe_collection[str(year_start) + '-' + str(month_start)]['text'].tolist()
    except KeyError:
        continue
        
    # Learn the vocabulary dictionary and return term-document matrix.
    counts = vectorizer_new.fit_transform(list_reviews_rest)
    
    #Transform a count matrix to a normalized tf-idf representation. 
    #(mainly =Terms with frequencies too hi or lo are removed)
    tfidf = TfidfTransformer().fit_transform(counts)
    
    #Save the data in matlab friendly format
    savemat('Inst500Matr' + str(ct), {'tfidf':tfidf})
    
    
    nmf = decomposition.NMF(n_components=n_topics)
    
    #We are looking for two matrices whose product approximates tfidf.
    
    # Learn the model (parameters etc.). I.e "take a look at what we have". returns nothing.
    W = nmf.fit(tfidf)
    
    # dictionary (factorization matrix)
    H = nmf.components_
    
    # Extracts the data and performs NMF on the tfidf-treated dataset. 
    # Returns the matrix W (the weight matrix), to be multiplied by the matrix H (dictionary) to get target tfidf.
    X2 = nmf.fit_transform(tfidf)
    ct=ct+1
    feature_names = vectorizer_new.get_feature_names()
    print month
    for i,text_topic in enumerate(X2.T):
        
        # i.e -> i, [content of column i of X2]
        # i-th topic named text_topic
        # Why sort them backwards?
        text_top = text_topic.argsort()[:-21:-1]
        word_top = nmf.components_[i].argsort()[:-11:-1]
        
        # For each text topic in the NMF resulting list, print the weight vector and the term vector.
        print [nmf.components_[i,k] for k in nmf.components_[i].argsort()[:-11:-1]]
        print [", ".join([feature_names[i] for i in word_top])]
        
        # print the amount of non-zero components in the weight vector
        l=[text_topic[i] for i in text_top]
        print np.count_nonzero(l)
            

In [ ]:
>>> import numpy as np
>>> X = np.array([[1, 1], [2, 1], [3, 1.2], [4, 1], [5, 0.8], [6, 1]])
>>> from sklearn.decomposition import NMF
>>> model = NMF(n_components=2, init='random', random_state=0)
>>> W = model.fit_transform(X)
>>> H = model.components_

In [ ]:
X

In [ ]:
for i, t in enumerate(X):
    print i,t

In [ ]:
print feature_names

In [ ]:
#save the feature names in a csv file
type(feature_names)
import io

f = io.open('test_inst_Torino.csv', 'w', encoding='utf8')
for item in feature_names:
    print item
    f.write("%s\n" % item)

In [ ]:
#Boundaries of the map
print (df_new['lonbin'].min()-step_lon/2., df_new['latbin'].min()-step_lat/2., df_new['lonbin'].max()+step_lon/2., df_new['latbin'].max()+step_lat/2)

In [ ]:
len(df_new_nodup_cells['n'].unique())
list_rc=df_new_nodup2_cells['n'].tolist()
with open('n_inst500.txt', 'w') as fp:
    fp.write('\n'.join('%s' % x for x in list_rc))

In [ ]:
list_rc

In [ ]:
df_new_nodup2_cells

In [ ]:
df_new_nodup2_cells.head()

In [ ]:
import time
import datetime
list_time=[]
for month in range(1,(len(list_years)-1)*12+2,3):
        year_start=list_years[0]+ int(month/12)
        month_start=month%12  
        year_end=list_years[0]+ int(month+11-0.5)/12
        month_end=1+(month+5-1)%12
             
     
        if month_end in([2,4,6,9,11]):
            day=30
        else:
            day=31
        deb=str(year_start) + '-' + str(month_end) + '-1'
        deb_new=time.mktime(datetime.datetime.strptime(deb, "%Y-%m-%d").timetuple())
        if month==1:
            list_time= np.append(deb_new,list_time)
        else:
            list_time=np.append(list_time,deb_new)


In [ ]:
print list_time

In [ ]:

with open('times_inst500.txt', 'w') as fp:
    fp.write('\n'.join('%s' % x for x in list_time))